In [1]:
import numpy as np
import pandas as pd
import keras
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Flatten
import os,pickle

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers.embeddings import Embedding




C:\Users\KAUSTUBH\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

data = pd.read_csv('./train.csv')
pd.set_option('display.max_colwidth',-1)


pos=data[data['Sentiment']=='POSTIVE']
mix=data[data['Sentiment']=='MIXED']
mix=mix.append(mix, ignore_index=True)
data=data.append(pos[:150], ignore_index=True)
data=data.append(mix, ignore_index=True)


#pos=data.loc[[data['Sentiment']=='POSITIVE']]
#pos=data.iloc(pos,Ignore_index='TRUE')

#pos=data.loc[[data['Sentiment']=='POSTIVE']]
#neg=data.loc[[data['Sentiment']=='NEUTRAL']]
#neu=data.loc[[data['Sentiment']=='NEGATIVE']]
#mix=data.loc[[data['Sentiment']=='MIXED']]

#print(len(pos))
#print(len(neg))
#print(len(neu))
#print(len(mix))

In [3]:
len(data)

1270

In [4]:
num_words = 250
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(data['Scene'].values)
X = tokenizer.texts_to_sequences(data['Scene'].values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
X = pad_sequences(X, maxlen=max_length_of_text)

Found 5735 unique tokens.


In [5]:
import pickle
with open('./tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
X[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [7]:
dic={
    'POSTIVE':0,
    'NEGATIVE':1,
    'MIXED':2,
    'NEUTRAL':3
}


In [8]:
y = data['Sentiment']

y = [  dic.get(y[index])   for index in range(len(y))  ]
y = to_categorical(y,num_classes=4)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1016, 200) (1016, 4)
(254, 200) (254, 4)


In [18]:
embed_dim = 25
lstm_out = 16
batch_size = 32

inputs = Input((max_length_of_text, ))
x = Embedding(num_words, embed_dim)(inputs)
x = LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.25)(x)
x = Dense(32,activation='relu')(x)
x = Dense(4,activation='softmax')(x)
model = Model(inputs, x)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 200, 25)           6250      
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                2688      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 132       
Total params: 9,614
Trainable params: 9,614
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [21]:
model.fit(X_train, y_train,validation_data=(X_test, y_test) , batch_size = batch_size, epochs = 35)

Train on 1016 samples, validate on 254 samples
Epoch 1/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.7277 - acc: 0.7136 - val_loss: 1.2478 - val_acc: 0.5197
Epoch 2/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.7055 - acc: 0.7343 - val_loss: 1.2445 - val_acc: 0.5354
Epoch 3/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.6871 - acc: 0.7333 - val_loss: 1.2702 - val_acc: 0.5118
Epoch 4/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.6751 - acc: 0.7303 - val_loss: 1.2720 - val_acc: 0.5079
Epoch 5/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.6739 - acc: 0.7333 - val_loss: 1.2606 - val_acc: 0.5236
Epoch 6/35
1016/1016 [==============================] - 5s 5ms/step - loss: 0.6433 - acc: 0.7421 - val_loss: 1.2946 - val_acc: 0.5433
Epoch 7/35
1016/1016 [==============================] - 6s 5ms/step - loss: 0.6483 - acc: 0.7382 - val_loss: 1.2908 - val_acc: 0.5236
Epoch 8/35
1016

In [22]:
model.save('model1_11.h5')

Found 400000 word vectors.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 50)           286800    
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 260       
Total params: 316,500
Trainable params: 29,700
Non-trainable params: 286,800
_________________________________________________________________
None


Epoch 1/15
1016/1016 [==============================] - 31s 31ms/step - loss: 1.3657 - acc: 0.3061
Epoch 2/15
1016/1016 [==============================] - 5s 4ms/step - loss: 1.3522 - acc: 0.3051
Epoch 3/15
1016/1016 [==============================] - 5s 4ms/step - loss: 1.3378 - acc: 0.3219
Epoch 4/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.3254 - acc: 0.3465
Epoch 5/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.3166 - acc: 0.3425
Epoch 6/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.3105 - acc: 0.3474
Epoch 7/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.3138 - acc: 0.3632
Epoch 8/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.3049 - acc: 0.3652
Epoch 9/15
1016/1016 [==============================] - 5s 5ms/step - loss: 1.2902 - acc: 0.3661
Epoch 10/15
1016/1016 [==============================] - 5s 4ms/step - loss: 1.2895 - acc: 0.3760
Epoch 11/15
1016/1016 [====

254/254 [==============================] - 11s 43ms/step
Score: 1.34
Validation Accuracy: 0.30


In [54]:
model.save('model1.h5')

In [55]:
model2.save('model2_1.h5')

In [56]:
data0 = pd.read_csv('sample_train.csv')
pd.set_option('display.max_colwidth',-1)
X1 = tokenizer.texts_to_sequences(data0['Scene'].values)
X1 = pad_sequences(X1, maxlen=max_length_of_text)

data1=pd.read_csv('sample_submission.csv')
pd.set_option('display.max_colwidth',-1)
y1 = data1['Sentiment']
y1=pd.get_dummies(y1)


5/5 [==============================] - 7s 1s/step
Score: 6.86
Validation Accuracy: 0.20


In [57]:
score1,acc1 = model2.evaluate(X1[:5], y1, batch_size = batch_size)
print("Score: %.2f" % (score1))
print("Validation Accuracy: %.2f" % (acc1))

5/5 [==============================] - 6s 1s/step
Score: 1.54
Validation Accuracy: 0.40


In [58]:
preds = model.predict(X1, batch_size=batch_size)

In [59]:
preds2=model2.predict(X1,batch_size=batch_size)

In [66]:
pred_classes1=model.predict_classes(X1,batch_size=batch_size)

AttributeError: 'Model' object has no attribute 'predict_classes'

In [60]:
preds=

array([[5.6950765e-04, 9.9933225e-01, 6.2570609e-05, 3.5629837e-05],
       [5.2252423e-04, 3.6594833e-03, 9.9438322e-01, 1.4347576e-03],
       [5.6831934e-04, 9.9926716e-01, 8.7644818e-05, 7.6847922e-05],
       [2.8943552e-05, 1.1350375e-03, 9.9862838e-01, 2.0755915e-04],
       [6.5349723e-04, 1.7391122e-04, 2.0622827e-04, 9.9896634e-01],
       [9.9109370e-01, 2.2560767e-04, 1.9415221e-04, 8.4865028e-03]],
      dtype=float32)

In [61]:
np.argmax(preds, axis=1)

array([1, 2, 1, 2, 3, 0], dtype=int64)

In [62]:
np.argmax(preds2,axis=1)

array([1, 2, 1, 1, 1, 1], dtype=int64)

In [64]:
data1['Sentiment']

0    NEUTRAL 
1    NEUTRAL 
2    MIXED   
3    POSTIVE 
4    NEGATIVE
Name: Sentiment, dtype: object

In [65]:
y1

,MIXED,NEGATIVE,NEUTRAL,POSTIVE
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,1,0,0


In [67]:
data0 = pd.read_csv('sample_train.csv')
pd.set_option('display.max_colwidth',-1)
data0.head()

,Index,Scene
0,1,"{Monica} How about, youre moving!! {Rachel} Look! This is ridiculous. We should be packing you!! {Phoebe} Hey, how are you guys doing? {Rachel} Great! Monicas moving! {Monica} I am not! {Rachel} Oh really?! Then how come all your stuff is in this box?! {Phoebe} Okay, you guys. You guys I think I know whats going on here. Okay, you guys {Monica} No Phoebe I am mad! {Phoebe} Well, deep-deep-deep down! {Rachel} Yeah, Im just mad! {Phoebe} Then keep running."
1,2,{Phoebe} Or we could just follow your clever jokes  any ideas?
2,3,"{Joey} Hey Rach! Hey, you mind if I read my comic books in here? {Rachel} Sure! Why? {Joey} Oh well, Chandler and Monica are over there and it's kinda hard to concentrate. {Rachel} What?! {Rachel} She just called and said that she was gonna be working late! {Rachel} She keeps lying to me! {Rachel} That's it! {Rachel} I'm just gonna go over there and confront them right now!"
3,4,"{Joey} You forget how many great songs Heart had. {Chandler} Yeah. {Ross} You know, Barracuda was the first song I learned to play on the keyboard. {Chandler} So, you heard it, you repeated it, so that must mean you wrote it. {Joey} Oh, you guys, with this joke. I gotta say, I know I cracked up, but Im not even sure I got it. {Ross} What, you didnt get it? The doctor is a monkey. {Chandler} And monkeys cant write out prescriptions. {Chandler} You are not allowed to laugh at my joke. {Ross} Your joke? Well, I think the Hef would disagree, which is why he sent me a check for one hundred ah-dollars."
4,5,"{Chandler} Hey Joe! You wanna shoot some hoops? {Joey} Oh no, I cant go. Im practicing; I got an audition to be the host of a new game show."


In [0]:
X1 = tokenizer.texts_to_sequences(data0['Scene'].values)
X1 = pad_sequences(X1, maxlen=max_length_of_text)

In [69]:
data1=pd.read_csv('sample_submission.csv')
pd.set_option('display.max_colwidth',-1)
y1 = data1['Sentiment']
y1=pd.get_dummies(y1)
score1,acc1 = model.evaluate(X1[:5], y1, batch_size = batch_size)
print("Score: %.2f" % (score1))
print("Validation Accuracy: %.2f" % (acc1))

5/5 [==============================] - 0s 5ms/step
Score: 1.45
Validation Accuracy: 0.20
